This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/shadow_deploy).

## Shadow Deployment Tutorial

Wallaroo provides a method of testing the same data against two different models or sets of models at the same time through **shadow deployments** otherwise known as **parallel deployments**.  This allows data to be submitted to a pipeline with inferences running on two different sets of models.  Typically this is performed on a model that is known to provide accurate results - the **champion** - and a model that is being tested to see if it provides more accurate or faster responses depending on the criteria known as the **challengers**.  Multiple challengers can be tested against a single champion.

As described in the Wallaroo blog post [The What, Why, and How of Model A/B Testing](https://www.wallaroo.ai/blog/the-what-why-and-how-of-a/b-testing):

> In data science, A/B tests can also be used to choose between two models in production, by measuring which model performs better in the real world. In this formulation, the control is often an existing model that is currently in production, sometimes called the champion. The treatment is a new model being considered to replace the old one. This new model is sometimes called the challenger....

> Keep in mind that in machine learning, the terms experiments and trials also often refer to the process of finding a training configuration that works best for the problem at hand (this is sometimes called hyperparameter optimization).

When a shadow deployment is created, only the inference from the champion is returned in the InferenceResult Object `data`, while the result data for the shadow deployments is stored in the InferenceResult Object `shadow_data`.

The following tutorial will demonstrate how:

* Upload champion and challenger models into a Wallaroo instance.
* Create a shadow deployment in a Wallaroo pipeline.
* Perform an inference through a pipeline with a shadow deployment.
* View the `data` and `shadow_data` results from the InferenceResult Object.
* View the pipeline logs and pipeline shadow logs.

This tutorial provides the following:

* `dev_smoke_test.json`:  Sample test data used for the inference testing.
* `models/keras_ccfraud.onnx`:  The champion model.
* `models/modelA.onnx`: A challenger model.
* `models/xgboost_ccfraud.onnx`: A challenger model.

All models are similar to the ones used for the Wallaroo-101 example included in the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials).

## Prerequisites

* A deployed Wallaroo instance
* The following Python libraries installed:
  * `os`
  * `json`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame

## Steps

### Import libraries

The first step is to import the libraries required.


In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import os
# For Wallaroo SDK 2023.1
os.environ["ARROW_ENABLED"]="True"

### Connect to Wallaroo

Connect to your Wallaroo instance and save the connection as the variable `wl`.

In [ ]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

### Set Variables

The following variables are used to create or use existing workspaces, pipelines, and upload the models.  Adjust them based on your Wallaroo instance and organization requirements.

In [4]:
workspace_name = 'ccfraudcomparisondemo'
pipeline_name = 'ccshadow'
champion_model_name = 'ccfraud-lstm'
champion_model_file = 'models/keras_ccfraud.onnx'
shadow_model_01_name = 'ccfraud-xgb'
shadow_model_01_file = 'models/xgboost_ccfraud.onnx'
shadow_model_02_name = 'ccfraud-rf'
shadow_model_02_file = 'models/modelA.onnx'

### Workspace and Pipeline

The following creates or connects to an existing workspace based on the variable `workspace_name`, and creates or connects to a pipeline based on the variable `pipeline_name`.

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [6]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline


name,ccshadow
created,2023-03-03 19:20:44.224839+00:00
last_updated,2023-03-03 19:20:44.224839+00:00
deployed,(none)
tags,
versions,32d751ec-f17b-4516-bc26-e3ef914f0938
steps,


### Load the Models

The models will be uploaded into the current workspace based on the variable names set earlier and listed as the `champion`, `model2` and `model3`.

In [7]:
champion = wl.upload_model(champion_model_name, champion_model_file).configure()
model2 = wl.upload_model(shadow_model_01_name, shadow_model_01_file).configure()
model3 = wl.upload_model(shadow_model_02_name, shadow_model_02_file).configure()

### Create Shadow Deployment

A shadow deployment is created using the `add_shadow_deploy(champion, challengers[])` method where:

* `champion`: The model that will be primarily used for inferences run through the pipeline.  Inference results will be returned through the Inference Object's `data` element.
* `challengers[]`: An array of models that will be used for inferences iteratively.  Inference results will be returned through the Inference Object's `shadow_data` element.

In [8]:
pipeline.add_shadow_deploy(champion, [model2, model3])
pipeline.deploy()

Waiting for deployment - this will take up to 45s .............. ok


name,ccshadow
created,2023-03-03 19:20:44.224839+00:00
last_updated,2023-03-03 19:20:45.086221+00:00
deployed,True
tags,
versions,"c97f777b-72f2-4a94-b2ea-a21bb33c032f, 32d751ec-f17b-4516-bc26-e3ef914f0938"
steps,ccfraud-lstm


### Run Test Inference

Using the data from `sample_data_file`, a test inference will be made.

For Arrow enabled Wallaroo instances the model outputs are listed by column.  The output data is set by the term `out`, followed by the name of the model.  For the default model, this is `out.dense_1`, while the shadow deployed models are in the format `out_{model name}.variable`, where `{model name}` is the name of the shadow deployed model.

For Arrow disabled environments, the output is from the Wallaroo InferenceResult object.### Run Test Inference

Using the data from `sample_data_file`, a test inference will be made.  As mentioned earlier, the inference results from the `champion` model will be available in the returned InferenceResult Object's `data` element, while inference results from each of the `challenger` models will be in the returned InferenceResult Object's `shadow_data` element.

In [9]:
sample_data_file = './smoke_test.df.json'
response = pipeline.infer_from_file(sample_data_file)
display(response)

,time,in.tensor,out.dense_1,check_failures,out_ccfraud-rf.variable,out_ccfraud-xgb.variable
0,2023-03-03 19:21:00.069,"[1.0678324729, 0.2177810266, -1.7115145262, 0.682285721, 1.0138553067, -0.4335000013, 0.7395859437, -0.2882839595, -0.447262688, 0.5146124988, 0.3791316964, 0.5190619748, -0.4904593222, 1.1656456469, -0.9776307444, -0.6322198963, -0.6891477694, 0.1783317857, 0.1397992467, -0.3554220649, 0.4394217877, 1.4588397512, -0.3886829615, 0.4353492889, 1.7420053483, -0.4434654615, -0.1515747891, -0.2668451725, -1.4549617756]",[0.0014974177],0,[1.0],[0.0005066991]


### View Pipeline Logs

With the inferences complete, we can retrieve the log data from the pipeline with the pipeline `logs` method.  Note that for **each** inference request, the logs return **one entry per model**.  For this example, for one inference request three log entries will be created.

In [10]:
pipeline.logs()

,time,in.tensor,out.dense_1,check_failures,out_ccfraud-rf.variable,out_ccfraud-xgb.variable
0,2023-03-03 19:21:00.069,"[1.0678324729, 0.2177810266, -1.7115145262, 0.682285721, 1.0138553067, -0.4335000013, 0.7395859437, -0.2882839595, -0.447262688, 0.5146124988, 0.3791316964, 0.5190619748, -0.4904593222, 1.1656456469, -0.9776307444, -0.6322198963, -0.6891477694, 0.1783317857, 0.1397992467, -0.3554220649, 0.4394217877, 1.4588397512, -0.3886829615, 0.4353492889, 1.7420053483, -0.4434654615, -0.1515747891, -0.2668451725, -1.4549617756]",[0.0014974177],0,[1.0],[0.0005066991]


### View Logs Per Model

Another way of displaying the logs would be to specify the model.

For Arrow enabled Wallaroo instances the model outputs are listed by column.  The output data is set by the term `out`, followed by the name of the model.  For the default model, this is `out.dense_1`, while the shadow deployed models are in the format `out_{model name}.variable`, where `{model name}` is the name of the shadow deployed model.

For arrow disabled Wallaroo instances, to view the inputs and results for the shadow deployed models, use the pipeline `logs_shadow_deploy()` method.  The results will be grouped by the inputs.

In [14]:
logs = pipeline.logs()
display(logs)

,time,in.tensor,out.dense_1,check_failures,out_ccfraud-rf.variable,out_ccfraud-xgb.variable
0,2023-03-03 19:21:00.069,"[1.0678324729, 0.2177810266, -1.7115145262, 0.682285721, 1.0138553067, -0.4335000013, 0.7395859437, -0.2882839595, -0.447262688, 0.5146124988, 0.3791316964, 0.5190619748, -0.4904593222, 1.1656456469, -0.9776307444, -0.6322198963, -0.6891477694, 0.1783317857, 0.1397992467, -0.3554220649, 0.4394217877, 1.4588397512, -0.3886829615, 0.4353492889, 1.7420053483, -0.4434654615, -0.1515747891, -0.2668451725, -1.4549617756]",[0.0014974177],0,[1.0],[0.0005066991]


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the system.

In [15]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,ccshadow
created,2023-03-03 19:20:44.224839+00:00
last_updated,2023-03-03 19:20:45.086221+00:00
deployed,False
tags,
versions,"c97f777b-72f2-4a94-b2ea-a21bb33c032f, 32d751ec-f17b-4516-bc26-e3ef914f0938"
steps,ccfraud-lstm
